# 🔍 X(Twitter) 凍結アカウントチェッカー

Googleスプレッドシートのアカウント一覧をチェックして、凍結・存在しないアカウントを自動で色付けします。

---

## 📋 使い方（3ステップ）

### 1️⃣ 初回のみ：設定を入力
下の「⚙️ 設定」セルでスプレッドシートのURLとシート名を入力してください。

### 2️⃣ 「▶️ すべて実行」をクリック
上のメニューから「ランタイム」→「すべてのセルを実行」を選択します。

### 3️⃣ Googleアカウントで認証
初回のみGoogleアカウントへのアクセス許可が求められます。「許可」をクリックしてください。

---

**⏱️ 所要時間の目安**: 100件で約5〜8分

## ⚙️ 設定（ここを編集してください）

In [ ]:
#@title 📝 スプレッドシートの設定 { display-mode: "form" }

#@markdown ### スプレッドシートのURL
#@markdown スプレッドシートを開いた時のURLをコピペしてください
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/xxxxxxxx/edit" #@param {type:"string"}

#@markdown ### シート名
#@markdown チェックしたいシートの名前（下のタブに表示されている名前）
SHEET_NAME = "シート1" #@param {type:"string"}

#@markdown ### アカウントIDが入っている列
#@markdown A列なら「A」、B列なら「B」と入力
ACCOUNT_COLUMN = "A" #@param {type:"string"}

#@markdown ### ヘッダー行をスキップする
#@markdown 1行目がタイトル行の場合はチェック
SKIP_HEADER = True #@param {type:"boolean"}

print("✅ 設定完了！次のセルに進んでください。")

## 🔧 準備（自動実行されます）

In [ ]:
#@title 📦 必要なライブラリをインストール { display-mode: "form" }
print("📦 ライブラリをインストール中...")
!pip install -q gspread google-auth selenium webdriver-manager
!apt-get update -qq
!apt-get install -qq chromium-browser chromium-chromedriver
print("✅ インストール完了！")

In [ ]:
#@title 🔐 Googleアカウントに接続 { display-mode: "form" }
from google.colab import auth
import gspread
from google.auth import default

print("🔐 Googleアカウントに接続中...")
print("　 ポップアップが表示されたら「許可」をクリックしてください")

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

print("✅ 接続完了！")

In [ ]:
#@title 📊 スプレッドシートを読み込み { display-mode: "form" }
import re

def extract_account_id(value):
    """様々な形式からアカウントIDを抽出"""
    if not value:
        return None
    
    value = value.strip()
    
    # URL形式: https://x.com/username または https://twitter.com/username
    url_match = re.search(r'(?:x\.com|twitter\.com)/([a-zA-Z0-9_]+)', value)
    if url_match:
        return url_match.group(1)
    
    # @付き形式
    if value.startswith('@'):
        return value[1:].strip()
    
    # 純粋なID（英数字とアンダースコアのみ）
    if re.match(r'^[a-zA-Z0-9_]+$', value):
        return value
    
    return None

# スプレッドシートIDを抽出
match = re.search(r'/d/([a-zA-Z0-9-_]+)', SPREADSHEET_URL)
if not match:
    raise ValueError("❌ スプレッドシートのURLが正しくありません")

spreadsheet_id = match.group(1)
print(f"📊 スプレッドシートを開いています...")

spreadsheet = gc.open_by_key(spreadsheet_id)
worksheet = spreadsheet.worksheet(SHEET_NAME)

# アカウント一覧を取得
col_num = ord(ACCOUNT_COLUMN.upper()) - ord('A') + 1
all_values = worksheet.col_values(col_num)

# ヘッダーをスキップ
start_row = 2 if SKIP_HEADER else 1

# アカウントIDを抽出
accounts = []
skipped = []
for i, v in enumerate(all_values[start_row - 1:]):
    row_num = i + start_row
    account_id = extract_account_id(v)
    if account_id:
        accounts.append((row_num, account_id, v))  # 元の値も保存
    elif v.strip():  # 空でないのにIDが抽出できなかった
        skipped.append((row_num, v))

print(f"✅ {len(accounts)} 件のアカウントを読み込みました")

# デバッグ: 最初の5件を表示
print("\n📋 読み込んだアカウントID（最初の5件）:")
for row, acc_id, original in accounts[:5]:
    print(f"   行{row}: {acc_id}")

if skipped:
    print(f"\n⚠️ {len(skipped)} 件のIDが認識できませんでした:")
    for row, val in skipped[:5]:
        print(f"   行{row}: '{val}'")

## 🚀 チェック実行

In [ ]:
#@title ▶️ 凍結チェックを開始 { display-mode: "form" }
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import time
import concurrent.futures

def create_driver():
    """Chromeドライバーを作成"""
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('--window-size=1280,720')
    options.add_argument('--lang=ja')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    
    service = Service('/usr/bin/chromedriver')
    return webdriver.Chrome(service=service, options=options)

def check_account(account_info):
    """単一アカウントをチェック（改善版）"""
    row, account_id, original = account_info
    url = f"https://x.com/{account_id}"
    
    driver = None
    try:
        driver = create_driver()
        driver.set_page_load_timeout(30)
        driver.get(url)
        
        # 十分な待機時間（4秒）
        time.sleep(4)
        
        content = driver.page_source.lower()  # 小文字に統一
        
        # 凍結判定（複数パターン）
        suspended_patterns = [
            'account suspended',
            'アカウントは凍結されています',
            'this account is suspended',
            'account is temporarily',
            'suspendedaccount',
        ]
        for pattern in suspended_patterns:
            if pattern.lower() in content:
                return (row, account_id, 'frozen', '凍結')
        
        # 存在しない判定（複数パターン）
        not_exist_patterns = [
            "this account doesn't exist",
            'このアカウントは存在しません',
            "hmm...this page doesn't exist",
            'try searching for something else',
            'ページは存在しません',
            'user not found',
            'ユーザーが見つかりません',
            'something went wrong',  # よくあるエラー表示
        ]
        for pattern in not_exist_patterns:
            if pattern.lower() in content:
                return (row, account_id, 'not_exist', '存在しない')
        
        # 正常なプロフィールの判定（複数の条件を AND で判定）
        # プロフィール要素が実際に存在するかチェック
        has_profile = False
        
        # 方法1: data-testidで確認
        if 'data-testid="primarycolumn"' in content:
            # ユーザー名表示があるか
            if 'data-testid="username"' in content or f'@{account_id.lower()}' in content:
                has_profile = True
        
        # 方法2: フォロー数表示で確認（日本語/英語両対応）
        if not has_profile:
            follow_patterns = [
                ('following', 'follower'),
                ('フォロー中', 'フォロワー'),
                ('フォロー', 'フォロワー'),
            ]
            for f1, f2 in follow_patterns:
                if f1.lower() in content and f2.lower() in content:
                    # さらにユーザー名も含まれているか確認
                    if f'@{account_id.lower()}' in content or account_id.lower() in content:
                        has_profile = True
                        break
        
        if has_profile:
            return (row, account_id, 'normal', '正常')
        else:
            # 判定不能（ページ読み込みが不完全か、予期しないレイアウト）
            return (row, account_id, 'unknown', '判定不能')
            
    except Exception as e:
        return (row, account_id, 'error', f'エラー')
    finally:
        if driver:
            try:
                driver.quit()
            except:
                pass

# メイン処理
print(f"🚀 チェック開始: {len(accounts)} 件")
print(f"⏱️ 開始時刻: {datetime.now().strftime('%H:%M:%S')}")
print("="*50)

check_results = []
frozen_count = 0
not_exist_count = 0
unknown_count = 0

# 並列処理（2件同時に減らして安定性向上）
CONCURRENCY = 2
with concurrent.futures.ThreadPoolExecutor(max_workers=CONCURRENCY) as executor:
    futures = {executor.submit(check_account, acc): acc for acc in accounts}
    
    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        result = future.result()
        check_results.append(result)
        
        row, account, status, reason = result
        if status == 'frozen':
            frozen_count += 1
            print(f"❄️ {account}: {reason}")
        elif status == 'not_exist':
            not_exist_count += 1
            print(f"👻 {account}: {reason}")
        elif status == 'unknown':
            unknown_count += 1
        
        # 進捗表示（10件ごと）
        if (i + 1) % 10 == 0 or i == len(accounts) - 1:
            pct = (i + 1) / len(accounts) * 100
            print(f"⏳ 進捗: {i + 1}/{len(accounts)} ({pct:.0f}%)")

print("="*50)
print(f"✅ チェック完了！")
print(f"⏱️ 終了時刻: {datetime.now().strftime('%H:%M:%S')}")
print(f"❄️ 凍結: {frozen_count} 件")
print(f"👻 存在しない: {not_exist_count} 件")
print(f"❓ 判定不能: {unknown_count} 件")

In [ ]:
#@title 📝 結果をスプレッドシートに書き込み { display-mode: "form" }
from datetime import datetime

print("📝 結果をスプレッドシートに書き込み中...")

# 色の定義
COLOR_FROZEN = {'red': 1, 'green': 0.8, 'blue': 0.8}  # 薄い赤
COLOR_NOT_EXIST = {'red': 0.85, 'green': 0.85, 'blue': 0.85}  # グレー
COLOR_UNKNOWN = {'red': 1, 'green': 1, 'blue': 0.8}  # 薄い黄色

# バッチ更新用のリクエストを作成
requests = []
col_idx = ord(ACCOUNT_COLUMN.upper()) - ord('A')

for row, account, status, reason in check_results:
    if status == 'frozen':
        color = COLOR_FROZEN
    elif status == 'not_exist':
        color = COLOR_NOT_EXIST
    elif status == 'unknown':
        color = COLOR_UNKNOWN
    else:
        continue  # 正常なアカウントはスキップ
    
    # 背景色を設定
    requests.append({
        'repeatCell': {
            'range': {
                'sheetId': worksheet.id,
                'startRowIndex': row - 1,
                'endRowIndex': row,
                'startColumnIndex': col_idx,
                'endColumnIndex': col_idx + 1
            },
            'cell': {
                'userEnteredFormat': {
                    'backgroundColor': color
                },
                'note': f"{reason} ({datetime.now().strftime('%Y/%m/%d %H:%M')})"
            },
            'fields': 'userEnteredFormat.backgroundColor,note'
        }
    })

# 一括更新
if requests:
    spreadsheet.batch_update({'requests': requests})
    print(f"✅ {len(requests)} 件のセルを更新しました")
else:
    print("ℹ️ 更新が必要なアカウントはありませんでした")

print("")
print("🎉 完了！スプレッドシートを確認してください。")
print(f"📊 {SPREADSHEET_URL}")

---

## 📖 よくある質問

### Q: エラーが出た場合は？
「ランタイム」→「ランタイムを再起動」してからもう一度実行してください。

### Q: 90分経つとどうなる？
Colabは90分で自動的に停止します。再度このページを開いて「すべて実行」してください。

### Q: 色の意味は？
- 🔴 薄い赤: 凍結されているアカウント
- ⚪ グレー: 存在しないアカウント
- 🟡 薄い黄色: 判定不能（要手動確認）
- 色なし: 正常なアカウント